In [147]:
import sqlite3

Creating a Database Named: food.db
with two tables: Menus, Restaurants

In [53]:
def create_tables(cur):
    cur.execute('DROP TABLE IF EXISTS Menus')
    cur.execute('DROP TABLE IF EXISTS Restaurants')
    cur.execute('CREATE TABLE Menus (menu_id INTEGER, food_item TEXT, price REAL)')
    conn.commit()
    cur.execute('CREATE TABLE Restaurants (restaurant_id INTEGER, restaurant_name TEXT, restaurant_location TEXT, menu_id INTEGER)')
    conn.commit()
    

In [150]:
conn = sqlite3.connect('food.db')
cur = conn.cursor()
create_tables(cur)
conn.commit()

In [151]:
cur.execute("SELECT name FROM sqlite_master")
result = cur.fetchall()
set(result) == set([('Menus',), ('Restaurants',)])

True

In [152]:
cur.execute("SELECT sql FROM sqlite_master")
result = cur.fetchall()
("(menu_id INTEGER, food_item TEXT, price REAL)" in result[0][0]) and \
("(restaurant_id INTEGER, restaurant_name TEXT, restaurant_location TEXT, menu_id INTEGER)" in result[1][0])

True

In [153]:
def read_csv(filename):
    h = 0
    c = 0
    data = {}
    k = []
    try:
        with open(filename, 'r') as fin:
             for line in fin:
                words = line.split(',')
                for w in words:
                    w = w.strip()
                    if h == 0:
                        data[w] = []
                        k.append(w)
                    else:
                        data[k[c]].append(w)
                        c += 1
                h = 1 
                c = 0
    except FileNotFoundError as e:
        print(f'Error: Cannot open file.{e}')
    except IOError as e:
        print(f'IO Error: Cannot open file.{e}')
    except PermissionError as e:
        print(f'Error: Cannot open file due to permissions. Need access rights.{e}')
    except Exception as e:
        print(f'Unexpected Error. {e}')

    return data   
    

In [154]:
import csv
filename = 'data.csv'
data = read_csv(filename)

In [155]:
len(data) == 4

True

In [156]:
set(["restaurant_name", "restaurant_location", "food_item", "price"]) == set(list(data.keys()))

True

In [157]:
len(data["restaurant_name"]) == len(data["restaurant_location"]) == len(data["food_item"]) == len(data["price"]) == 11


True

Insert data in to their respective tables. There should be 3 rows in the Restaurants table, one for each restaurant and 11 rows in the Menus table, one for each food item. **DO NOT INSERT THESE MANUALLY.** Please use some sort of loop to iterate through the data you have read in, constructing the needed tuples of information that should be added to the respective table.
* *If you cannot figure out how to loop through the data, I will still give you points if you insert things manually and pass the below tests. However, you will get no points for Implementation*


**An Example Row in Restaurants Table:**

(0, "Bob's Burgers", 'Netflix Studios', 0)

**An Example Row in Menus Table:**

(1, 'scone', 3.8)

Useful functions:
* execute

RELEVANT SQL Keywords:
* INSERT INTO
* VALUES

In [158]:
'''Assuming that the records in csv are loaded and formatted like our sample, where restaurants and their
menu items are listed consecutiviely in order by restaurant. This algorithm is comparing the current record with previous
record to see if there is a new restaurant being listed, so it relies on this assumption.
'''

i = 0                                #i is index for position in the data dictionary's values list 
f = 0                                #f is tracking the uniqe number of restaurants
new_flag = False                     #new_flag is set to True only when a new restaurant has been detected so that duplicated records are not inserted for the same restaurant
max = len(data['food_item'])         #the number of columns in the data file (aka list items in dictionary values)
prev = data['restaurant_name'][0]    #prev stores the previous restaurant. Initializing it to the first restaurant
sql = 'INSERT OR IGNORE INTO Restaurants (restaurant_id, restaurant_name, restaurant_location, menu_id) VALUES(?,?,?,?)'   
sql_menu = 'INSERT OR IGNORE INTO Menus (menu_id, food_item, price) VALUES(?,?,?)'   

for i in range(max):                 #loop through each column in file(aka list items in dictionary values)
    for d in data:                   #loop through all dict keys
        if d == 'restaurant_name':   #for each dict key, set the current value
            name = data[d][i]
            if name != prev:
                f += 1               #if new restaurant, increase count of unique restaurants
                new_flag = True      #set flag for a new restaurant detected
            prev = data[d][i]        #store for next loop
        if d == 'restaurant_location':
            loc = data[d][i]
        if d == 'food_item':
            food = data[d][i]
        if d == 'price':
            price = data[d][i]
   
    if new_flag == False:            #insert new restaurant when it IS a new restaurant
        cur.execute(sql,(f, name, loc, f,))
        conn.commit()
    cur.execute(sql_menu,(f, food, price,))  #insert menu items for every itme
    conn.commit()
    i += 1                           #increment position counter
    new_flag = False                 #initialize flag for each loop of dict keys / new row in file  

In [159]:
cur.execute('SELECT * FROM Restaurants')
results = cur.fetchall()
set(results) == set([(0, "Bob's Burgers", 'Netflix Studios', 0), 
                     (1, 'Central Perk', '199 Lafayette Street', 1), 
                     (2, 'Tire Shop Taqueria', '4077 S Avalon Blvd', 2)])

True

In [160]:
cur.execute('SELECT * FROM Menus')
results = cur.fetchall()
set(results) == set([(0, 'burger', 5.45), 
                     (0, 'fries', 2.75), 
                     (0, 'milkshake', 10.98), 
                     (0, 'grilled cheese', 4.15), 
                     (1, 'coffee', 3.28), 
                     (1, 'muffin', 1.75), 
                     (1, 'scone', 3.8), 
                     (2, 'taco', 2.0), 
                     (2, 'quesadilla', 4.0), 
                     (2, 'mulitas', 3.57), 
                     (2, 'jarritos', 2.5)])

True

In [161]:
conn.close()